In [1]:
%load_ext watermark
import pandas as pd
import numpy as np
from scipy.stats import dirichlet
import logging

import matplotlib.pyplot as plt
import matplotlib as mpl
import matplotlib.colors
from matplotlib.colors import LinearSegmentedColormap, ListedColormap
import seaborn as sns


from myst_nb import glue
from IPython.display import display, Markdown

from scipy.stats import halfnorm, multinomial
import gridforecast as gfcast

# available data

columns =  [
    'sample_id',
    'code',
    'quantity',
    'pcs/m',
    'feature_name',
    'location',
    'parent_boundary',
    'city', 
    'canton',
    'feature_type',
    'date'
]


import logging

logging.basicConfig(
    filename='app.log', 
    level=logging.DEBUG,
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s'
)

logger = logging.getLogger(__name__)

def create_jeffreys_prior_matrix(index_range, categories, epsilon=0.01):
    # Initialize the matrix
    jeffreys_prior_matrix = np.zeros((len(index_range), len(categories)))
    
    # Calculate Jeffreys prior values using the modified formula
    for i, x in enumerate(index_range):
        prior = 1 / (x + epsilon)  # Adding epsilon to avoid division by zero
        # Assign this value to all categories for this index
        jeffreys_prior_matrix[i, :] = prior
    
    return jeffreys_prior_matrix

(gridforecaster)=
# Grid forecast

We consider that forecasting or predicting is the process of making statements about events that have yet to occurr. In this case we are using historical results to form our opinion about the probability of an event in the future. The event we considering is rather pedestrian:

> What will I find at the beach today, given what has been found at __other similar__ beaches or what was found at the beach in the past ?

Which means that our estimation of the objects we are likely to find is based on our own experience as well as the experience of others under similar conditions. This reasoning does not tell us about the reason why the object has bypassed the elaborate system put in place to prevent it from ending up on the beach. 

We do, however, identify quantifiable vectors that certainly contain the cause. The vectors come directly from the official topographical map for the territory. As a result locations can be grouped according to the magnitude of the vector that has been derived from the topographical map. By comparing local results to national results with a quantifiable vector we have a very efficient way to combine experiences beyond a geographic limit. Thus we can estimate the comparison from one location using the results from locations that have similar attributes but not geographically related.

Here we show how the `gridforecast` module works. Specifically the different methods of the  `gridforecast.MulitnomialDirichlet` class. It is a grid aproximation in Bayesien framework. We start the process with conditional probability, make the connection with Bayes` theorem and finish with the conjugate relationship Dirichlet-Multinomial. The application to survey data is implemented in scipy and numpy. 

Our first consideration, however, is wether or not our research question is comensurate with our assumptions of the model. Otherwise no amount of mathematical manipulation will persuade a reasonable individual that an outstanding or extreme result is probable when there is no evidence that rises to the same level.

## Assumptions of the model about the sample data

The data is assumed to be subject to the experience of the surveyor and each survey is independent and identically distributed. We add to these basic assumptions the particularities of the domain:

1. Locations that have similar environmental conditions will yield similar survey results
2. There is an exchange of material (trash) between the beach and body of water
3. Following from two, the material recovered at the beach is a result of the assumed exchange
4. The type of activities adjacent to the survey location are an indicator of the trash that will be found there
5. Following from four and three, the local environmental conditions are an indicator of the local contribution to the mix of objects at the beach
6. Surveys are not accurate
   * Some objects will be misidentified
   * Not all objects will be found
   * There will be inaccuracies in object counts or data entry

**Following 1 through 6:** the survey results are a reasonable estimate of the minimum number of objects that were present at the time the survey was completed. 


## Conditional probability

[Conditional probability](https://en.wikipedia.org/wiki/Conditional_probability) is a fundamental concept in probability theory that describes the probability of an event occurring given that another event has already occurred. It is denoted as $P(A∣B)$, which reads as "the probability of A given B". 

The conditional probability of event \(A\) given event \(B\) is defined as:

$$P(A|B) = \frac{P(A \cap B)}{P(B)}$$

where:

- $P(A|B)$ is the conditional probability of $A$ given $B$.
- $P(A \cap B)$ is the joint probability of both $A$ and $B$ occurring.
- $P(B)$ is the probability of event $B$ occurring, provided that $P(B) > 0$.


### Bayes' theorem

[Bayes' Theorem](https://en.wikipedia.org/wiki/Bayes%27_theorem) is a fundamental principle in probability theory and statistics that describes the probability of an event based on __prior__ knowledge of conditions that might be related to the event. It allows for the updating of probabilities as new evidence or information becomes available. It is derived from the definition of conditional probability.

__Deriving Bayes theorem__

::::{grid} 2 2 2 2
:gutter: 1

:::{grid-item-card} Define conditional probability

For events \(A\) and \(B\):

$$P(A|B) = \frac{P(A \cap B)}{P(B)}$$

$$P(B|A) = \frac{P(A \cap B)}{P(A)}$$

:::

:::{grid-item-card} The joint probability $P(A \cap B)$

From the first equation:

$$P(A \cap B) = P(A|B) \cdot P(B)$$

From the second equation:

$$P(A \cap B) = P(B|A) \cdot P(A)$$

:::

:::{grid-item-card} Equate the two expressions

$$P(A|B) \cdot P(B) = P(B|A) \cdot P(A)$$

Solve for $P(A|B)$

$$P(A|B) = \frac{P(B|A) \cdot P(A)}{P(B)}$$

:::

:::{grid-item-card} This is Bayes' Theorem

$$P(A|B) = \frac{P(B|A) \cdot P(A)}{P(B)}$$

:::
::::

#### Prior knowledge

In the context of Bayes' theorem, the term "prior" refers to the prior probability, which is the probability of an event or hypothesis before any new evidence or data is taken into account. It represents the initial degree of belief in a particular outcome based on existing knowledge or assumptions.

> In this use case the __prior__ is what we __beleive__ we will find at the beach, before we get to the beach, given everything we know about beaches and litter in Switzerland. Our beliefs are based on the cumulative experience from all previous visits to the beach, or beaches that are similar. Our beliefs come from what we have actually experienced.

Mathematically, if we are trying to determine the probability of a hypothesis A given new evidence B, the prior probability is denoted as P(A). It is the baseline probability of A before considering the new evidence provided by B.

Bayes' Theorem uses the prior probability along with the likelihood of the evidence given the hypothesis and the marginal probability of the evidence to update the probability of the hypothesis. This updated probability is called the posterior probability.

### Empirical Bayes

Empirical Bayes methods are statistical techniques that combine the principles of Bayesian inference with empirical data. These methods use data to estimate the prior distribution, which is then used in the Bayesian framework to update probabilities and make inferences. Using this method means that our prior distribution is testable in the sense of Jaynes

> A piece of information _I_ concerning a parameter $\theta$ will be called __testable__ if, given any proposed prior probability assignment  $f( \theta )$ $d\theta$, there is a procedure which will determine unambiguously whether $f( \theta )$ does or does not agree with the information _I_. ([Jaynes,1968](https://bayes.wustl.edu/etj/articles/prior.pdf))

In traditional Bayesian analysis, the prior distribution is chosen based on subjective beliefs or historical data. In contrast, empirical Bayes methods __estimate the prior distribution directly from the observed data, making the process more objective and often more practical in large-scale problems__. ([Petrone, S. et al, 2014](https://link.springer.com/article/10.1007/s40300-014-0044-1))

#### Conjugate prior

In Bayesian statistics, a [conjugate prior](https://en.wikipedia.org/wiki/Conjugate_prior) is a prior distribution that, when combined with a given likelihood through Bayes' theorem, results in a [posterior distribution](https://en.wikipedia.org/wiki/Posterior_probability) of the same family as the prior. This property simplifies the computation of the posterior distribution.

1. Jaynes, E.T.: ["Probability Theory: The Logic of Science"](https://bayes.wustl.edu/etj/prob/book.pdf): Emphasized the logical consistency and practical advantages of conjugate priors.
2. Gelman, A. et al.: ["Bayesian Data Analysis"](http://www.stat.columbia.edu/~gelman/book/) : discusses conjugate priors in the context of hierarchical models and practical Bayesian analysis.

#### Deriving conjugate relationship

The [binomial](https://en.wikipedia.org/wiki/Binomial_distribution), [multinomial](https://en.wikipedia.org/wiki/Multinomial_distribution), and [Dirichlet](https://en.wikipedia.org/wiki/Dirichlet_distribution) distributions are intrinsically linked through the concept of conjugate priors in Bayesian statistics. The binomial distribution describes the probability of a fixed number of successes in a series of independent trials, with a success probability _p_. When modeling this in a Bayesian framework, the Beta distribution is used as a conjugate prior for _p_. This means that the posterior distribution, after observing data, remains a Beta distribution, simplifying the update process.

Extending this to multiple categories, the multinomial distribution generalizes the binomial by modeling the counts of outcomes across multiple categories. The Dirichlet distribution serves as the conjugate prior for the multinomial distribution, just as the Beta distribution does for the binomial. When using a Dirichlet prior, the posterior distribution after observing data also remains a Dirichlet distribution.

::::{grid} 2 2 2 2
:gutter: 1

:::{grid-item-card} Binomial Likelihood:

The binomial distribution models the number of successes in _n_ trials, given a success probability _p_:

$$P(X = k | p) = \binom{n}{k} p^k (1 - p)^{n - k}$$

The Beta distribution is a conjugate prior for the binomial likelihood, parameterized by $\alpha$ and $\beta$:

$$P(p | \alpha, \beta) = \frac{p^{\alpha - 1} (1 - p)^{\beta - 1}}{B(\alpha, \beta)}$$
:::

:::{grid-item-card} Posterior Distribution:

Combining the likelihood and prior using Bayes' theorem gives the posterior distribution:

$$P(p | k, n) \propto p^k (1 - p)^{n - k} \cdot p^{\alpha - 1} (1 - p)^{\beta - 1}$$

$$P(p | k, n) \propto p^{k + \alpha - 1} (1 - p)^{n - k + \beta - 1}$$

Which is a Beta distribution:

$$P(p | k, n) = \text{Beta}(p | k + \alpha, n - k + \beta)$$
:::

:::{grid-item-card} Generalize binomial to multinomial

The multinomial distribution generalizes the binomial to more than two categories. For counts 
$\mathbf{x} = (x_1, x_2, \ldots, x_K) \quad \text{in} \quad K \quad \text{categories, given probabilities} \quad \mathbf{p} = (p_1, p_2, \ldots, p_K)$:

$$P(\mathbf{x} | \mathbf{p}) = \frac{n!}{x_1! x_2! \cdots x_K!} p_1^{x_1} p_2^{x_2} \cdots p_K^{x_K}$$

where: $n = \sum_{i=1}^K x_i$
:::

:::{grid-item-card} The conjugate prior to the multinomial

The Dirichlet distribution is a conjugate prior for the multinomial distribution, parameterized by α=(α1​,α2​,…,αK​):

$P(\mathbf{p} | \boldsymbol{\alpha}) = \frac{1}{B(\boldsymbol{\alpha})} \prod_{i=1}^K p_i^{\alpha_i - 1}$

where B(α) is the multivariate Beta function.
:::

:::{grid-item-card} Posterior Distribution
The posterior distribution is a combination of the likelihood and prior:

$$P(\mathbf{p} | \mathbf{x}) \propto \left( \prod_{i=1}^K p_i^{x_i} \right) \left( \prod_{i=1}^K p_i^{\alpha_i - 1} \right)$$

Which is a Dirichlet distribution:

$P(\mathbf{p} | \mathbf{x}) \propto \prod_{i=1}^K p_i^{x_i + \alpha_i - 1}$

:::
:::{grid-item-card} Posterior with updated parameters

$$P(\mathbf{p}|\mathbf{x}) = \text{D}(\mathbf{p}|x_1 + \alpha_1, \ldots, x_K + \alpha_K)$$

Where D is a _Dirichlet_ distribution
:::
::::

#### Grid Approximation

Grid approximation is a technique used in numerical analysis and statistical inference to approximate the values of a continuous function or parameter by evaluating it at a discrete set of points. This involves creating a grid of possible values within a defined range and calculating the function or parameter at each grid point. We are using $P(\mathbf{p}|\mathbf{x}) = \text{D}(\mathbf{p}|x_1 + \alpha_1, \ldots, x_K + \alpha_K)$ to approximate the grid.

### Empirical Bayes grid approximation using a conjugate prior

Empirical Bayes grid approximations involves estimating the prior and posterior distributions of parameters using a discretized set of values. In the context of the multinomial-Dirichlet conjugate relationship, this method is particularly effective. The Dirichlet distribution serves as the prior for the multinomial likelihood, and empirical Bayes methods estimate this prior directly from the data. By defining a grid of possible parameter values, in this case spaced every 0.1 units from 0 to the maximum observed value or 100, the posterior distribution is approximated by evaluating the likelihood and updating the Dirichlet prior at each grid point.

This approach simplifies the computational complexity of Bayesian inference. Instead of integrating over a continuous parameter space, which can be analytically challenging, grid approximation transforms the problem into a finite summation. The [multinomial-Dirichlet conjugate pair](https://en.wikipedia.org/wiki/Dirichlet-multinomial_distribution) ensures that the posterior remains in the Dirichlet family, making the updates straightforward.

__Adding probability__

We have noticed that using this method is sensitive to the max value of the likelihood. The further out the maximum is the more likely elevated values appear. This may be the case, but there is also the chance that extreme elements are just that.  

## The grid forecaster

The _grid forecaster_ refers to the methods defined in `gridforecast.py`. The main purpose of the _grid forecaster_ is to implement the process described in the preceding sections. Here we define three methods for making a forecast for a particular region that are inline with the assumptions.

__Given the experience in the past__

1. The prior probability is based on the collection of all previous experiences in the region
2. The prior probability is based on the 99th percentile of all previous experiences in the region

__Given the experience in similar locations__

3. The prior probability is based on the collection of experiences from locations with similar land use

### Predict density from past experiences

For regions that have a sampling history, previous to the most recent set of survey data, a forecast can be implemented by designating or selecting all the data in the region prior to the most recent survey results.

__Example creating reports and forecasts__

```python
# collecting the default data
data = session_config.collect_survey_data()

# the likelihood: the dates of the most recent samples
recent_dates = {'start':'2020-01-01', 'end':'2021-12-31'}
# the prior: the dates prior to the most recent samples
prior_dates = {'start':'2015-11-15', 'end':'2019-12-31'}
# the region of interest
canton = 'Vaud'

# the search parameters for the prior and likelihood
likelihood_params = {'canton':canton, 'date_range':recent_dates}
prior_params = {'canton':canton, 'date_range':prior_dates}

# verify the parameters exist in the data
# checking the parameters will verify that the requested data
# exists. If the query is possible it is executed and the value of
# comments='ok', if not empty arrays are returned with the message
# 'no survey results found'. The method returns the query data, a list
# of the sample locations and the comment.
likelihood_data, likelihood_locations, likelihood_comments = check_params(likelihood_params, data, logger)
prior_data, prior_locations, prior_comments = check_params(prior_params, data, logger)

# if there is data for both the likelihood and the prior
# make a survey report and a land use report for both sets of data
likelihood_report, likelihood_land_use = make_report_objects(likelihood_data)
prior_report, prior_land_use = make_report_objects(likelihood_data)

# make forecast from all the available liklihood data
forecast_object = MulitnomialDirichlet('comb', prior_report.sample_results['pcs/m'], likelihood_report.sample_results['pcs/m'], logger)

# make forecast limiting the likelihood to the 99the percentile
posterior_counts, comments = posterior_dirichlet_counts(lkl, prr, max_range=0.99)

# forecasts from all the data
forecasted_samples = forecast_object.sample_posterior()
forecasted_summary = forecast_object.get_descriptive_statistics()

# forecasts limited to the 99th percentile
sample_values_99, posterior_99, summary_99 = gfcast.dirichlet_posterior(posterior_counts)
```

### Predict density given similar locations

To predict density given similar locations we need to create another prior experience, that does not include any of the locations that are in the likelihood. Furthermore, we need to ensure that the land-use conditions in the likelihood are reflected in the prior.

```python
# determine the proportion of each land-use feature in the likelihood
weights = land_use_weights(likelihood_land_use, session_config.feature_variables)

# from the pool of available data select records that are not included in the likelihood
# in this case we eliminate the canton of interest, limit the date to the end date of the prior
# and create a survey report and land use report for *the other prior data*
other_data = data[(data.canton != canton)&(data['date'] <= prior_dates['end'])].copy()
other_prior_report, other_prior_land_use = gfcast.make_report_objects(other_prior_data)

# using the weights from the likelihood and the other_prior_land_use
other_prior_data, prior_weights = select_prior_data_by_feature_weight(other_prior_land_use, weights, session_config.feature_variables)
posterior_by_weight, weighted_comments = posterior_dirichlet_counts(likelihood_data, g['pcs/m'].values)
posterior_sample_values, weighted_dist, weighted_summary = dirichlet_posterior(posterior_by_weight)

```
### Different priors = different points of view

The three models are based off the experiences from the field and for the most part all give a reasonable estimate of what we would expect to find. The differences between the expected outcomes may or may not be of importance. We won't know untill we sample again.

## Using the grid forecast

In [2]:
import session_config
import reports
import geospatial
import userdisplay as disp
import gridforecast as gfcast

# collecting the default data
data = session_config.collect_survey_data()
data = data.reset_index()

# the likelihood: the dates of the most recent samples
recent_dates = {'start':'2020-01-01', 'end':'2021-12-31'}
# the prior: the dates prior to the most recent samples
prior_dates = {'start':'2015-11-15', 'end':'2019-12-31'}
# the region of interest
canton = 'Vaud'

# the search parameters for the prior and likelihood
likelihood_params = {'canton':canton, 'date_range':recent_dates}
prior_params = {'canton':canton, 'date_range':prior_dates}

# verify the parameters exist in the data
# checking the parameters will verify that the requested data
# exists. If the query is possible it is executed and the value of
# comments='ok', if not empty arrays are returned with the message
# 'no survey results found'. The method returns the query data, a list
# of the sample locations and the comment.
likelihood_data, likelihood_locations, likelihood_comments = gfcast.check_params(likelihood_params, data, logger)
prior_data, prior_locations, prior_comments = gfcast.check_params(prior_params, data, logger)

# if there is data for both the likelihood and the prior
# make a survey report and a land use report for both sets of data
likelihood_report, likelihood_land_use = gfcast.make_report_objects(likelihood_data)
prior_report, prior_land_use = gfcast.make_report_objects(prior_data)

# make forecast from all the available liklihood data
forecast_object = gfcast.MulitnomialDirichlet('comb', prior_report.sample_results['pcs/m'], likelihood_report.sample_results['pcs/m'], logger)

# make forecast limiting the likelihood to the 99the percentile
posterior_counts, comments = gfcast.posterior_dirichlet_counts(likelihood_report.sample_results['pcs/m'], prior_report.sample_results['pcs/m'], max_range=0.99)

# forecasts from all the data
forecasted_samples = forecast_object.sample_posterior()
forecasted_summary = forecast_object.get_descriptive_statistics()

# forecasts limited to the 99th percentile
sample_values_99, posterior_99, summary_99 = gfcast.dirichlet_posterior(posterior_counts)

### The grid size

The grid size for each combination is based on the maximum value of either the likelihood or the prior. 

In [3]:
forecast_object.compute_grid()

array([0.000e+00, 1.000e-02, 2.000e-02, ..., 7.707e+01, 7.708e+01,
       7.709e+01])

### The counts

The number of times that a survey result was either equal to zero or any other place on the grid can be accessed with `forecastobject.prior` or `forecastobject.compute_counts(forecast_object.prior_data)`

In [4]:
forecast_object.prior_data

array([0, 0, 0, ..., 0, 0, 1])

### The posterior parameters

The parameters for the Dirichlet posterior: `forecastobject.compute_posterior_params(self)` 

In [15]:
forecast_object.compute_posterior_params()

array([0.01, 0.01, 0.01, ..., 0.01, 0.01, 1.  ])

### The posterior distribution

The posterior distribution : `forecastobject.posterior_dist` is a `scipy.stats.dirichlet` object.

In [17]:
forecast_object.posterior_dist.mean()

array([3.26850793e-05, 3.26850793e-05, 3.26850793e-05, ...,
       3.26850793e-05, 3.26850793e-05, 3.26850793e-03])

### Sample the posterior distribution

Sample the posterior distribution

In [18]:
forecast_object.sample_posterior()

array([ 0.1 ,  0.47,  0.71,  0.99,  1.16,  1.26,  1.65,  1.86,  1.86,
        1.89,  1.89,  2.03,  2.1 ,  2.1 ,  2.21,  2.21,  2.21,  2.21,
        2.29,  2.29,  2.56,  2.56,  2.6 ,  2.68,  2.75,  2.75,  2.77,
        3.11,  3.19,  3.27,  3.35,  3.59,  3.73,  3.73,  3.81,  3.81,
        4.  ,  4.  ,  4.  ,  4.17,  4.22,  4.42,  4.55,  4.59,  5.12,
        5.26,  5.34,  5.36,  5.43,  5.59,  5.8 ,  6.04,  6.04,  6.13,
        6.15,  6.25,  6.59,  6.88,  6.97,  6.97,  7.26,  8.38,  8.92,
        9.45,  9.45,  9.45, 10.07, 10.08, 10.67, 12.36, 15.01, 15.01,
       16.36, 17.52, 17.52, 17.52, 17.89, 17.89, 18.36, 18.36, 18.97,
       22.38, 23.18, 23.5 , 23.65, 23.65, 23.73, 23.73, 38.69, 41.85,
       50.06, 53.78, 60.34, 60.61, 61.06, 61.06, 61.06, 62.89, 64.4 ,
       75.67])

### The 90% interval of the predictions

The 90% interval of the predictions : `forecast_object.compute_percentiles()`

In [19]:
forecast_object.compute_percentiles()

array([ 0.589,  2.77 ,  6.26 , 13.75 , 59.73 ])

In [13]:
prior_report.sampling_results_summary

{'total': 44911,
 'nsamples': 142,
 'average': 8.371830985915492,
 'quantiles': array([ 0.812 ,  2.6925,  4.905 ,  9.63  , 28.09  ]),
 'std': 10.433750262241018,
 'max': 77.1,
 'start': Timestamp('2015-11-23 00:00:00'),
 'end': Timestamp('2019-10-24 00:00:00')}

In [14]:
t = forecast_object.prior 
print(len(t))
print(sum(t))


7710
142
